In [14]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
    
base_url = 'https://theartsdesk.com/opera/reviews?page='
reviews=[]

In [ ]:
for page in range(85):

    if page%5==0:
        print(page)
        
    url = base_url + str(page)
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")
    listings = soup.find(attrs={"class": "view-content"}).find_all('td')

    for listing in listings:
        
        review = []        
        author = listing.find(attrs={"class": "author"}).get_text().strip()
        date   = listing.find(attrs={"class": "postDate"}).get_text().strip()
        date   = datetime.strptime(date, '%A, %d %B %Y').date()     
        url    = 'https://theartsdesk.com' + listing.find('h2').find('a').get('href')
        title  = listing.find('h2').find('a').get_text().strip()
        try:
            work = title.split(',')[0]
        except:
            work = ''
            
        try:
            venue = title.split(',')[1]
            venue = venue.split('-')[0].strip()            
            venue = venue.replace('review', '').strip()
        except:
            venue = ''
            
        r     = requests.get(url)
        soup  = BeautifulSoup(r.text, "html5lib")        
        text  = soup.find(attrs={"class": "articleContent"}).find_all('p')
        full_text=''
        for w in text:
            if w.get('class')==None:
                full_text += w.get_text()

        review.append(title)
        review.append(work)
        review.append(venue)
        review.append(date)        
        review.append(author)   
        review.append(full_text)
        review.append(url)        
        reviews.append(review)
        
import pandas as pd                     
headers = ['title', 'work', 'venue', 'date', 'author', 'full_text', 'url']
df = pd.DataFrame(reviews, columns=headers)
df.to_csv('../../data/processed/reviews/artsdesk.csv')